In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
from os.path import join

# 1. Exploratory Analysis
### We will visualize the data, in order to understand its structure & distribution, and build a model that makes sense

### The dataset is comprised of 1599 red wines and 4898 white wines, split into 2 csv files. We will first merge the 2 datasets and include a binary variable is_red

In [ ]:
DIR = "input" # location of input data

In [ ]:
data_red = pd.read_csv(join(DIR, "winequality-red.csv"), delimiter=";")
data_white = pd.read_csv(join(DIR, "winequality-white.csv"), delimiter=";")
data_red["is_red"] = 1
data_white["is_red"] = 0

data = pd.concat((data_red, data_white))

### Let's find correlations between dimensions, to potentially remove irrelevant ones

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(data.corr(),annot=True)

Some relatively strong correlations:
* free and total sulfur dioxides (expected since total depends on free)
* sulfur dioxides and is_red (also expected as that partly defines red & white wines)

Nothing really surprising here. Some of these variables may potentially be dropped if found to be irrelevant

### Let's now check if there are notable differences between red & white wine qualities.
### Here's the distribution of wine quality by wine type

In [ ]:
data.hist(column="quality", by=data["is_red"], density=True, range=(0,9))

We can see the disributions are similar between wine qualities, with most wines being mediocre (as expected) <br>
Looks normally distributed.

### Let's make sure our dataset is balanced by looking at possible outliers.
To do so, we will look at the boxplot of each dimension with respect to wine quality. <br>
It will give us an idea of the range and variability of features, as well their possible outliers <br>

The boxplot will split values in quartiles delimited by horizontal lines. Points far away from can be considered outliers.


In [ ]:
boxplots = plt.figure(figsize=(12,24))
for i, dim in enumerate(data): 
    if i < 10:  # no need for boxplot of wine type or wine qualities
        boxplots.add_subplot(5, 2, i+1)
        sns.boxplot(x="quality", y=dim, data=data, hue="is_red")        

Some categories (like fixed & volatile acidities) have many points located above the interquartile range, but not by much. Most variables have a few noticeable outliers.

Residual sugars, sulphates and chlorides have relatively high variance, by looking at their interquartile ranges.

# 2. Preprocessing
### We will now get our data ready for prediction, and try to optimize it

In [ ]:
# Set seed for reproducibility
SEED = 123

In [ ]:
# Separate target from dimensions, and remove high-variance columns
X_red = data_red.copy().drop(columns=["quality", "is_red", "chlorides", "residual sugar", "sulphates"])
y_red = data_red["quality"]

X_white = data_white.copy().drop(columns=["quality", "is_red", "chlorides"])
y_white = data_white["quality"]

In [ ]:
# Drop missing values, if any
if sum(data.isna().sum()) == 0:
    print("No missing values")

### Rescale data

In [ ]:
from sklearn.preprocessing import RobustScaler # resistant to data that has few large outliers
X_red = RobustScaler().fit_transform(X_red)
X_white = RobustScaler().fit_transform(X_white)

### PCA decomposition for removal of redundant features


In [ ]:
from sklearn.decomposition import PCA
X_red_PCA = PCA(0.95, random_state=SEED).fit_transform(X_red) # reduce dimensionality while keeping 95% of explained variance
X_white_PCA = PCA(0.95, random_state=SEED).fit_transform(X_white) # reduce dimensionality while keeping 95% of explained variance

print("PCA removed", X_red.shape[1] - X_red_PCA.shape[1], "dimensions from the red wine dataset")
print("PCA removed", X_white.shape[1] - X_white_PCA.shape[1], "dimensions from the white wine dataset")

# 3. Logistic Regression
### We will run a regularized logistic regression in both the original and reduced input spaces

In [ ]:
n_jobs = -1 # Set this to the max. number of CPU cores to be used (-1 for all)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures

def logistic_regression(X_input, y_input, degrees=[1]):
    accuracies = [-1]*(max(degrees)+1)
    best_predictions = best_conf_matrix = None

    for degree in degrees:
        clf = LogisticRegressionCV(Cs=[1, 1/10, 1/100, 1/1000, 1/10000, 1/100000], # Test different C values, equal to 1/lambda
                               cv=4,
                               max_iter=100000 if max(degrees) >= 4 else 1000,
                               n_jobs=n_jobs,
                               multi_class="multinomial",
                               random_state=SEED
                              )
        X_input = PolynomialFeatures(degree).fit_transform(X_input)
        X_train, X_test, y_train, y_test = train_test_split(X_input, y_input,
                                                    stratify=y_input/len(y_input), # stratify to preserve distribution
                                                    random_state=SEED
                                                   )                
        clf.fit(X_train, y_train)

        predictions = clf.predict(X_test)
        conf_matrix = confusion_matrix(y_test, predictions)
        acc = accuracy_score(y_test, predictions)
        if acc > max(accuracies):
            best_predictions, best_conf_matrix = predictions, conf_matrix
        accuracies[degree] = acc

    return best_predictions, best_conf_matrix, accuracies

In [ ]:
# Here are the results you should get after running the regression
# We show these so you know the best polynomial degree for each dataset

acc_X_red = [-1, 0.5825, 0.5725, 0.57, 0.555]
acc_X_red_PCA = [-1, 0.565, 0.5775, 0.56, 0.555]
acc_X_white = [-1, 0.5371428571428571, 0.566530612244898, 0.583673469, 0.6]
acc_X_white_PCA = [-1, 0.5338775510204081, 0.5624489795918367, 0.572244898, 0.582857143]
degrees = [1, 2, 4, 4]

# Feel free to change the degrees argument for the function calls below

pred_X_red, conf_X_red, acc_X_red = logistic_regression(X_red, y_red, [degrees[0]])
pred_X_PCA_red, conf_X_PCA_red, acc_X_red_PCA = logistic_regression(X_red_PCA, y_red, [degrees[1]])
pred_X_white, conf_X_white, acc_X_white = logistic_regression(X_white, y_white, [degrees[2]])
pred_X_PCA_white, conf_X_PCA_white, acc_X_white_PCA = logistic_regression(X_white_PCA, y_white, [degrees[3]])

Let's compare the accuracies

In [ ]:
title = "Regression accuracy on red wines"
print(title)
print("-"*len(title))
print("Before PCA:", round(max(acc_X_red)*100, 2), "%.", "Polynomial degree", acc_X_red.index(max(acc_X_red)))
print("After PCA:", round(max(acc_X_red_PCA)*100, 2), "%.", "Polynomial degree", acc_X_red_PCA.index(max(acc_X_red_PCA)))
print()
mode = y_red.mode().sum()
print("Accuracy of constant predictor:", round((y_red == mode).sum()*100/len(y_red), 2), "%")

random_pred = np.random.permutation(y_red)
print("Accuracy of random chance:", round((y_red == random_pred).sum()*100/len(y_red), 2), "%")

In [ ]:
title = "Regression accuracy on white wines"
print(title)
print("-"*len(title))
print("Before PCA:", round(max(acc_X_white)*100, 2), "%.", "Polynomial degree", acc_X_white.index(max(acc_X_white)))
print("After PCA:", round(max(acc_X_white_PCA)*100, 2), "%.", "Polynomial degree", acc_X_white_PCA.index(max(acc_X_white_PCA)))
print()
mode = y_white.mode().sum()
print("Accuracy of constant predictor:", round((y_white == mode).sum()*100/len(y_white), 2), "%")

random_pred = np.random.permutation(y_white)
print("Accuracy of random chance:", round((y_white == random_pred).sum()*100/len(y_white), 2), "%")

Not impressive.

PCA has brought little to no value. This could be due to its linear nature. <br>
Polynomial mappings also did not improve performance by much

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
confs = plt.figure(figsize=(16, 6))
confs.add_subplot(1, 2, 1)
plot_confusion_matrix(conf_X_red, classes=np.unique(y_red), title="Confusion matrix for red wines")
confs.add_subplot(1, 2, 2)
plot_confusion_matrix(conf_X_white, classes=np.unique(y_white), title="Confusion matrix for white wines")

# Extra: Concatenating the datasets & re-constructing the target
### We saw that the value of quality is mainly split into 4 values. Let's see if grouping would help performance

In [ ]:
unique, counts = np.unique(data["quality"], return_counts=True)
print("Distribution of wine quality")
dict(zip(unique, counts*100.0/len(data)))

We will group 3 & 4, and 7-9 into a new column

In [ ]:
data_new = data.copy()
data_new["quality"] = 1*(data["quality"] >= 3) + 1*(data["quality"] >= 5) + 1*(data["quality"] >= 6) + 1*(data["quality"] >= 7)

In [ ]:
unique, counts = np.unique(data_new["quality"], return_counts=True)
print("New distribution of wine quality")
dict(zip(unique, counts*100.0/len(data)))

Let's now try our algorithm on this dataset.

In [ ]:
X_new, y_new = data.loc[:, data_new.columns != "quality"], data_new["quality"]
pred_new, conf_new, acc_new = logistic_regression(X_new, y_new)

In [ ]:
print("Accuracy of logistic regression after re-grouping quality:", round(max(acc_new)*100, 2), "%")

No improvement